In [1]:
import pandas as pd
from gensim.models import Word2Vec
import gensim
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import spacy
import string
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
#combine above and this
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
from scipy.stats import kendalltau

In [3]:
#delete this after and replace with reviews.csv
!git clone https://github.com/dregmi08/Milestone-2-Data-Exploration-Initial-Preprocessing.git



Cloning into 'Milestone-2-Data-Exploration-Initial-Preprocessing'...
remote: Enumerating objects: 118, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 118 (delta 56), reused 38 (delta 13), pack-reused 0 (from 0)
Receiving objects: 100% (118/118), 44.74 MiB | 11.59 MiB/s, done.
Resolving deltas: 100% (56/56), done.


## Preprocessing and generating Word2Vec

In [4]:
#df = pd.read_csv("reviews.csv")
df = pd.read_csv('Milestone-2-Data-Exploration-Initial-Preprocessing/reviews.csv')

In [5]:
df.head()

,Time_submitted,Review,Rating,Total_thumbsup,Reply
0,2022-07-09 15:00:00,"Great music service, the audio is high quality...",5,2,NaN
1,2022-07-09 14:21:22,Please ignore previous negative rating. This a...,5,1,NaN
2,2022-07-09 13:27:32,"This pop-up ""Get the best Spotify experience o...",4,0,NaN
3,2022-07-09 13:26:45,Really buggy and terrible to use as of recently,1,1,NaN
4,2022-07-09 13:20:49,Dear Spotify why do I get songs that I didn't ...,1,1,NaN


In [6]:
def clean_text(text):
    tokens = word_tokenize(text)
    lower = [token.lower() for token in tokens]
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in lower]
    words = [word for word in stripped if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    porter = PorterStemmer()
    words = [porter.stem(word) for word in words]
    non_stop_words = [w for w in words if not w in stop_words]
    return " ".join(non_stop_words)

In [7]:
df["Review_cleaned"] = df["Review"].apply(clean_text)

In [15]:
def categorize_rating(rating):
    if rating < 3:
        return "NEGATIVE"
    else:
        return "POSITIVE"

In [16]:
df["Rating_category"] = df["Rating"].apply(categorize_rating)

In [13]:
from transformers import pipeline
from tqdm import tqdm

tqdm.pandas()
sentiment_pipeline = pipeline("sentiment-analysis")

df['Sentiment'] = df['Review_cleaned'].progress_apply(lambda x: sentiment_pipeline(x)[0]['label'])



No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
100%|██████████| 61594/61594 [1:15:34<00:00, 13.58it/s]


In [17]:
print("Sentiment Frequencies: ")
print("Predicted: ")
print(df['Sentiment'].value_counts())
print("By Rating (< 3 = Negative, else Positive): ")
print(df['Rating_category'].value_counts())
print("\nAccuracy: ")
print(sum([1 if pred == actual else 0 for pred, actual in zip(df['Sentiment'], df['Rating_category'])]) / df.shape[0])

Sentiment Frequencies: 
Predicted: 
Sentiment
NEGATIVE    41088
POSITIVE    20506
Name: count, dtype: int64
By Rating (< 3 = Negative, else Positive): 
Rating_category
POSITIVE    36823
NEGATIVE    24771
Name: count, dtype: int64

Accuracy: 
0.6860570834821573


In [18]:
df.to_csv('reviewsSentiment.csv')